# odinson
Retrieve original Marvel Comics editions for stories published in old brazilian comics

##### 2021.11.14

---

## Souces

[Guia dos Quadrinhos](http://www.guiadosquadrinhos.com/)
- [Heróis da TV](http://www.guiadosquadrinhos.com/edicao/herois-da-tv-2-serie-n-1/htv0302/6274)
- [Superaventuras Marvel](http://www.guiadosquadrinhos.com/edicao/superaventuras-marvel-n-1/sam0301/7927)
- [Almanaque Premiere Marvel](http://www.guiadosquadrinhos.com/edicao/almanaque-premiere-marvel-n-1/apm02001/18611)

---


## 1. Develop retrieving:
- edition
    - id="nome_titulo_lb"
- ~cover image? link?~
    - id="ampliar_capa"
- story (portuguese)
    - all: id="texto_pag_detalhe"
- main character
- original publication
- year
- original title

In [2]:
htvurl = "http://www.guiadosquadrinhos.com/edicao/herois-da-tv-2-serie-n-1/htv0302/6274"

### 1.1 dev scrape main #1 page

In [3]:
# dev scraping
import requests
from bs4 import BeautifulSoup

URL = htvurl
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [5]:
# Edition
soup.find(id="nome_titulo_lb")

<span id="nome_titulo_lb">Heróis da TV <nobr>2ª Série -</nobr> <nobr>n° 1</nobr></span>

In [6]:
# Cover
soup.find(id="ampliar_capa")

<a alt="Heróis da TV n° 1 - Abril" href="http://www.guiadosquadrinhos.com/edicao/ShowImage.aspx?id=6274&amp;path=abril/h/htv0302001.jpg" id="ampliar_capa" title="Heróis da TV n° 1 - Abril"><img alt="Heróis da TV n° 1 - Abril" height="619" src="http://www.guiadosquadrinhos.com/edicao/ShowImage.aspx?id=6274&amp;path=abril/h/htv0302001.jpg&amp;w=400&amp;h=619" title="Heróis da TV n° 1 - Abril" width="400"/></a>

In [ ]:
# All stories (page section)
results = soup.find(id="texto_pag_detalhe")
# print(results.prettify())
results

In [10]:
# All stories (individual)
stories = results.find_all("div", class_="historia")
for story in stories:
    print(story, end="\n"*2)

<div class="historia">O Devorador de Planetas</div>

<div class="historia">Deve um deus prevalecer?</div>

<div class="historia">Duelo de ferro</div>

<div class="historia">Amargo adeus</div>

<div class="historia">Meia-Noite traz a morte!</div>

<div class="historia">A chama e a fúria</div>



In [11]:
stories

[<div class="historia">O Devorador de Planetas</div>,
 <div class="historia">Deve um deus prevalecer?</div>,
 <div class="historia">Duelo de ferro</div>,
 <div class="historia">Amargo adeus</div>,
 <div class="historia">Meia-Noite traz a morte!</div>,
 <div class="historia">A chama e a fúria</div>]

In [23]:
# Take the contents for 1 story
# results
# stories[0:2]
story = f"{results}".split(f"{stories[1]}")[1].split(f"{stories[2]}")[0]
print(story)


<br/><strong>Personagens: </strong> <a href="../../../personagem/thor-(thor-odinson-(donald-blake-jake-olson))/14">Thor</a>,  <a href="../../../personagem/galactus-(galan-)/105">Galactus</a>,  <a href="../../../personagem/ego-o-planeta-vivo/348">Ego, O Planeta Vivo</a>,  <a href="../../../personagem/registrador-211/5695">Registrador 211</a>
<br/><strong>Roteiro: </strong> <a href="../../../artista/stan-lee/2">Stan Lee</a> <br/><strong>Desenho: </strong> <a href="../../../artista/jack-kirby/1">Jack Kirby</a> <br/><strong>Arte-Final: </strong> <a href="../../../artista/vince-colletta/46">Vince Colletta</a> <br/><strong>Editor original: </strong> <a href="../../../artista/stan-lee/2">Stan Lee</a>
<br/>Publicada originalmente em   <a href="../../../edicao-estrangeira/thor-(1966)-n-161/261/5918">Thor (1966) n° 161</a>/1969 - Marvel Comics
             
<br/>Título original americano: "Shall a God Prevail?". Confronto final entre Thor e as forças de Galactus.


               


### 1.2 retrieve list of links for all editions